In [1]:
import gym
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
import numpy as np
import random

Using TensorFlow backend.


In [2]:
EPISODES = 1000

In [3]:
class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000) # create a deque of max length 2000 for the experience replay buffer
        self.gamma = 0.95 # discount rate
        self.epsilon = 1.0 # exploration rate
        self.epsilon_min = 0.01 
        self.epsilon_decay = 0.995
        self.learning_rate = 0.001 # learning rate for the optimizer
        self.model = self.__build_model()
    
    # fuction that returns the deep Q learning network model
    def __build_model(self):
        # Neural net for Deep Q network modelling
        model = Sequential()
        model.add(Dense(24,input_dim = self.state_size,activation='relu'))
        model.add(Dense(24,activation='relu'))
        model.add(Dense(self.action_size,activation='linear'))
        model.compile(loss='mse',optimizer = Adam(lr=self.learning_rate))
        return model
    
    def remember(self,state,action,reward,next_state,done):
        self.memory.append((state,action,reward,next_state,done))
    
    # predict the action
    def act(self,state):
        # epsilon-greedy method. with prob eps/k select an action to explore
        if np.random.rand()<=self.epsilon:
            return random.randrange(self.action_size)
        act_values = self.model.predict(state)
        return np.argmax(act_values[0]) # why the [0] ? check later
    
    def replay(self,batch_size):
        minibatch = random.sample(self.memory,batch_size)
        for state,action,reward,next_state,done in minibatch:
            if done:
                target = reward
            if not done:
                target = reward+self.gamma*np.amax(self.model.predict(next_state)[0])
            target_f = self.model.predict(state)
            target_f[0][action] = target
            self.model.fit(state,target_f,epochs=1,verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay
    
    
    # load weights of the model from file
    def load(self,name):
        self.model.load_weights(name)
    
    def save(self,name):
        self.model.save_weights(name)
        
    
if __name__ == "__main__":
    env = gym.make('CartPole-v1')
    state_size = env.observation_space.shape[0]
    action_size = env.action_space.n
    agent = DQNAgent(state_size,action_size)
    done = False
    batch_size = 64
    
    for e in range(EPISODES):
        state = env.reset()
        state = np.reshape(state,[1,state_size])
        for time in range(500):
            env.render()
            action = agent.act(state)
            next_state,reward,done,_= env.step(action)
            reward = reward if not done else -10
            next_state = np.reshape(next_state,[1,state_size])
            agent.remember(state,action,reward,next_state,done)
            state = next_state
            if done:
                print("episode: {}/{}, score: {}, e: {:.2}"
                      .format(e, EPISODES, time, agent.epsilon))
                break
            if len(agent.memory)>batch_size:
                agent.replay(batch_size)
        if e%10==0:
            agent.save("cartpole-v1-dqn.h5")
    

episode: 0/1000, score: 13, e: 1.0
episode: 1/1000, score: 13, e: 1.0
episode: 2/1000, score: 12, e: 1.0
episode: 3/1000, score: 9, e: 1.0
episode: 4/1000, score: 15, e: 0.99
episode: 5/1000, score: 13, e: 0.93
episode: 6/1000, score: 11, e: 0.88
episode: 7/1000, score: 12, e: 0.83
episode: 8/1000, score: 31, e: 0.71
episode: 9/1000, score: 37, e: 0.59
episode: 10/1000, score: 18, e: 0.54
episode: 11/1000, score: 9, e: 0.51
episode: 12/1000, score: 9, e: 0.49
episode: 13/1000, score: 11, e: 0.46
episode: 14/1000, score: 8, e: 0.45
episode: 15/1000, score: 11, e: 0.42
episode: 16/1000, score: 8, e: 0.41
episode: 17/1000, score: 28, e: 0.35
episode: 18/1000, score: 41, e: 0.29
episode: 19/1000, score: 48, e: 0.23
episode: 20/1000, score: 80, e: 0.15
episode: 21/1000, score: 72, e: 0.11
episode: 22/1000, score: 113, e: 0.06
episode: 23/1000, score: 99, e: 0.036
episode: 24/1000, score: 100, e: 0.022
episode: 25/1000, score: 107, e: 0.013
episode: 26/1000, score: 184, e: 0.01
episode: 27/1

KeyboardInterrupt: 